In [2]:
import sys
sys.path.append('./src/')

In [3]:
import time
import copy
import h5py
import glob
import numpy as np
import tensorflow as tf
import src.cameras as cameras
from tensorflow.python.client import device_lib
from src.mix_den_model import LinearModel
from src.data_utils import postprocess_3d, define_actions, project_to_cameras
from src.data_utils import transform_world_to_camera, postprocess_3d, normalization_stats, normalize_data
from src.data_utils import unNormalizeData

## Flags

In [4]:
FLAGS = {}
FLAGS["learning_rate"] = 1e-3 # "Learning rate"
FLAGS["dropout"] = 0.5 # "Dropout keep probability. 1 means no dropout"
FLAGS["batch_size"] = 64 #"batch size to use during training")
FLAGS["epochs"] = 200 # "How many epochs we should train for")
FLAGS["camera_frame"] = True # "Convert 3d poses to camera coordinates")
FLAGS["max_norm"] = True # "Apply maxnorm constraint to the weights")
FLAGS["batch_norm"] = True # "Use batch_normalization")

# Data loading
FLAGS["predict_14"] = False # "predict 14 joints")
FLAGS["use_sh"] = True # "Use 2d pose predictions from StackedHourglass")
FLAGS["action"] = "All" # "The action to train on. 'All' means all the actions")

# Architecture
FLAGS["linear_size"] = 1024 # "Size of each model layer.")
FLAGS["num_layers"] = 2 # "Number of layers in the model.")
FLAGS["residual"] = True # "Whether to add a residual connection every 2 layers")

# Evaluation
FLAGS["procrustes"] = False # "Apply procrustes analysis at test time")
FLAGS["evaluateActionWise"] = True # "The dataset to use either h36m or heva")

# Directories
FLAGS["cameras_path"] = "../data/h36m/cameras.h5" # "Directory to load camera parameters")
FLAGS["data_dir"] = "../data/h36m/" # "Data directory")
FLAGS["train_dir"] = "../experiments/test_git/" # "Training directory.")
FLAGS["load_dir"] = "../Models/mdm_5_prior/" # "Specify the directory to load trained model")

# Train or load
FLAGS["sample"] = False # "Set to True for sampling.")
FLAGS["test"] = False # "Set to True for sampling.")
FLAGS["use_cpu"] = False # "Whether to use the CPU")
FLAGS["load"] = 3192601 # "Try to load a previous checkpoint.")
FLAGS["miss_num"] = 1 # "Specify how many missing joints.")

### 4679232 for mdm_5
### 4338038 for mdm prior

# Misc
FLAGS["use_fp16"] = False # "Train using fp16 instead of fp32.")
FLAGS["n_joints"] = 16

In [5]:
FLAGS["load_dir"] = "experiments/first_May13/"

summaries_dir = os.path.join(FLAGS["train_dir"], "summary")

## Data Utils

In [5]:
def normalization_stats(complete_data):

    data_mean = np.mean(complete_data, axis=0)
    data_std  = np.std(complete_data, axis=0)

    return data_mean, data_std

def normalize_data(data, mu, stddev):

    if data.shape[-1]//FLAGS['n_joints'] == 3:
        new_data = data
        new_data[:, 3:] = np.divide((data[:, 3:] - mu[3:]), stddev[3:] )
        return new_data

    return np.divide( (data - mu), stddev )

In [5]:
def unNormalizeData(normalized_data, data_mean, data_std):
    T = normalized_data.shape[0] # Batch size
    D = data_mean.shape[0] # Dimensionality

    orig_data = np.zeros((T, D), dtype=np.float32)

    orig_data[:, :] = normalized_data

    # Multiply times stdev and add the mean
    stdMat = data_std.reshape((1, D))
    stdMat = np.repeat(stdMat, T, axis=0)
    meanMat = data_mean.reshape((1, D))
    meanMat = np.repeat(meanMat, T, axis=0)
    orig_data[:, 3:] = np.multiply(orig_data[:, 3:], stdMat[:, 3:]) + meanMat[:, 3:]
    return orig_data

In [6]:
def load_h36m_dataset(bpath, subjects, actions, dim):

    indices_to_select = [0, 1, 2, 3, 6, 7, 8, 12, 13, 15, 17, 18, 19, 25, 26, 27]

    if not dim in [2,3]:
        raise(ValueError, 'dim must be 2 or 3')

    data = {}

    for subj in subjects:
        for action in actions:

            print('Reading subject {0}, action {1}'.format(subj, action))

            dpath = os.path.join( bpath, 'S{0}'.format(subj), 'MyPoses/{0}D_positions'.format(dim), '{0}*.h5'.format(action) )
            print( dpath )

            fnames = glob.glob( dpath )

            loaded_seqs = 0
            for fname in fnames:
                seqname = os.path.basename( fname )

                # This rule makes sure SittingDown is not loaded when Sitting is requested
                if action == "Sitting" and seqname.startswith( "SittingDown" ):
                    continue

                # This rule makes sure that WalkDog and WalkTogeter are not loaded when
                # Walking is requested.
                if seqname.startswith( action ):
                    print( fname )
                    loaded_seqs = loaded_seqs + 1

                with h5py.File( fname, 'r' ) as h5f:
                    poses = np.array(h5f['{0}D_positions'.format(dim)][:])
                    # poses = poses.reshape((32, 3, poses.shape[1]))
                    # poses = poses[indices_to_select, :, :]
                    # poses = poses.reshape((48, poses.shape[2]))

                poses = poses.T
                data[ (subj, action, seqname) ] = poses

        if dim == 2:
            assert loaded_seqs == 8, "Expecting 8 sequences, found {0} instead".format( loaded_seqs )
        else:
            assert loaded_seqs == 2, "Expecting 2 sequences, found {0} instead".format( loaded_seqs )

    return data

## Load Model

In [7]:
def create_model(session, batch_size):

    model = LinearModel(
      FLAGS["linear_size"],
      FLAGS["num_layers"],
      FLAGS["residual"],
      FLAGS["batch_norm"],
      FLAGS["max_norm"],
      batch_size,
      FLAGS["learning_rate"],
      summaries_dir,
      FLAGS["predict_14"],
      dtype=tf.float16 if FLAGS["use_fp16"] else tf.float32)

    ckpt = tf.train.get_checkpoint_state(FLAGS["load_dir"], latest_filename="checkpoint")

    if ckpt and ckpt.model_checkpoint_path:
        if FLAGS["load"] > 0:
            if os.path.isfile(os.path.join(FLAGS["load_dir"],"checkpoint-{0}.index".format(FLAGS["load"]))):
                ckpt_name = os.path.join( os.path.join(FLAGS["load_dir"],"checkpoint-{0}".format(FLAGS["load"])) )
            else:
                raise ValueError("Asked to load checkpoint {0}, but it does not seem to exist".format(FLAGS["load"]))
        else:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        
        model.saver.restore(session, ckpt.model_checkpoint_path)
        return model
    else:
        raise( ValueError, "Checkpoint {0} does not seem to exist".format( ckpt.model_checkpoint_path ) )

In [8]:
def evaluate_batches( sess, model, data_mean_3d, data_std_3d, data_mean_2d, data_std_2d,
    encoder_inputs, decoder_outputs, current_epoch=0 ):

    nbatches = len( encoder_inputs )
    n_joints = 16

    # Loop through test examples
    all_dists, start_time, loss = [], time.time(), 0.0
    all_poses_3d = []
    all_enc_in =[]

    for i in range(nbatches):

        if current_epoch > 0 and (i+1) % log_every_n_batches == 0:
            print("Working on test epoch {0}, batch {1} / {2}".format( current_epoch, i+1, nbatches) )

        enc_in, dec_out = encoder_inputs[i], decoder_outputs[i]
        # enc_in = data_utils.generage_missing_data(enc_in, FLAGS.miss_num)
        dp = 1.0 # dropout keep probability is always 1 at test time
        step_loss, loss_summary, out_all_components_ori = model.step( sess, enc_in, dec_out, dp, isTraining=False )
        loss += step_loss

        out_all_components = np.reshape(out_all_components_ori,[-1, model.HUMAN_3D_SIZE+2, model.num_models])
        out_mean = out_all_components[:, : model.HUMAN_3D_SIZE, :]


        # denormalize
        enc_in  = unNormalizeData( enc_in,  data_mean_2d, data_std_2d)
        enc_in_ = copy.deepcopy(enc_in)
        all_enc_in.append(enc_in_)
        dec_out = unNormalizeData( dec_out, data_mean_3d, data_std_3d )
        pose_3d = np.zeros((enc_in.shape[0], 48, out_mean.shape[-1]))

        for j in range(out_mean.shape[-1]):
            pose_3d[:, :, j] = unNormalizeData( out_mean[:, :, j], data_mean_3d, data_std_3d )

        pose_3d_ = copy.deepcopy(pose_3d)
        all_poses_3d.append(pose_3d_)

        dec_out = dec_out[:, 3:]
        pose_3d = pose_3d[:, 3:, :]

        assert dec_out.shape[0] == FLAGS["batch_size"]
        assert pose_3d.shape[0] == FLAGS["batch_size"]

        # Compute Euclidean distance error per joint
        sqerr = (pose_3d - np.expand_dims(dec_out,axis=2))**2 # Squared error between prediction and expected output
        dists = np.zeros((sqerr.shape[0], n_joints, sqerr.shape[2])) # Array with L2 error per joint in mm

        for m in range(dists.shape[-1]):
            dist_idx = 0
            for k in np.arange(0, n_joints*3, 3):
                # Sum across X,Y, and Z dimenstions to obtain L2 distance
                dists[:,dist_idx, m] = np.sqrt( np.sum( sqerr[:, k:k+3,m], axis=1 ))

                dist_idx = dist_idx + 1

        all_dists.append(dists)
        assert sqerr.shape[0] == FLAGS["batch_size"]

    step_time = (time.time() - start_time) / nbatches
    loss      = loss / nbatches

    all_dists = np.vstack( all_dists )
    aver_minerr = np.mean(np.min(np.sum( all_dists, axis=1),axis=1))/n_joints

    return aver_minerr, step_time, loss

In [9]:
device_count = {"GPU": 0}

sess = tf.Session(config=tf.ConfigProto(device_count=device_count, allow_soft_placement=True))

model = create_model(sess, FLAGS["batch_size"])






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
dim is deprecated, use axis instead

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead





Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from experiments/first_May13/checkpoint-3192601


In [9]:
def evaluate(data_2d, data_3d, means, stds):

    data_mean_3d, data_std_3d = means[1], stds[1]
    data_mean_2d, data_std_2d = means[0], stds[0]

    # === Create the model ===
    print("Creating %d bi-layers of %d units." % (FLAGS["num_layers"], FLAGS["linear_size"]))
    model.train_writer.add_graph( sess.graph )

    cum_err = 0           # select the mixture model which has mininum error
    encoder_inputs, decoder_outputs = model.get_all_batches( data_2d, data_3d, FLAGS["camera_frame"], training=False)

    act_err, step_time, loss = evaluate_batches(sess, model,
                                                data_mean_3d, data_std_3d,
                                                data_mean_2d, data_std_2d,
                                                encoder_inputs, decoder_outputs)

    print(act_err)

### Original Evaluate

In [10]:
def get_action_subset( poses_set, action ):
  """
  Given a preloaded dictionary of poses, load the subset of a particular action

  Args
    poses_set: dictionary with keys k=(subject, action, seqname),
      values v=(nxd matrix of poses)
    action: string. The action that we want to filter out
  Returns
    poses_subset: dictionary with same structure as poses_set, but only with the
      specified action.
  """
  return {k:v for k, v in poses_set.items() if k[1] == action}

In [35]:
def evaluate_batches_orig( sess, model,
  data_mean_3d, data_std_3d, dim_to_use_3d, dim_to_ignore_3d,
  data_mean_2d, data_std_2d, dim_to_use_2d, dim_to_ignore_2d,
  current_step, encoder_inputs, decoder_outputs, current_epoch=0 ):
  """
  Generic method that evaluates performance of a list of batches.
  May be used to evaluate all actions or a single action.

  Args
    sess
    model
    data_mean_3d
    data_std_3d
    dim_to_use_3d
    dim_to_ignore_3d
    data_mean_2d
    data_std_2d
    dim_to_use_2d
    dim_to_ignore_2d
    current_step
    encoder_inputs
    decoder_outputs
    current_epoch
  Returns

    total_err
    joint_err
    step_time
    loss
  """

  n_joints = 17 if not(FLAGS["predict_14"]) else 14
  nbatches = len( encoder_inputs )


  # Loop through test examples
  all_dists, start_time, loss = [], time.time(), 0.
  log_every_n_batches = 100
  all_poses_3d = []
  all_enc_in =[]

  for i in range(nbatches):

    if current_epoch > 0 and (i+1) % log_every_n_batches == 0:
      print("Working on test epoch {0}, batch {1} / {2}".format( current_epoch, i+1, nbatches) )

    enc_in, dec_out = encoder_inputs[i], decoder_outputs[i]
    # enc_in = data_utils.generage_missing_data(enc_in, FLAGS.miss_num)
    dp = 1.0 # dropout keep probability is always 1 at test time
    step_loss, loss_summary, out_all_components_ori = model.step( sess, enc_in, dec_out, dp, isTraining=False )
    loss += step_loss

    out_all_components = np.reshape(out_all_components_ori,[-1, model.HUMAN_3D_SIZE+2, model.num_models])
    out_mean = out_all_components[:, : model.HUMAN_3D_SIZE, :]


    # denormalize
    enc_in  = unNormalizeData( enc_in,  data_mean_2d, data_std_2d, dim_to_ignore_2d )
    enc_in_ = copy.deepcopy(enc_in)
    all_enc_in.append(enc_in_)
    dec_out = unNormalizeData( dec_out, data_mean_3d, data_std_3d, dim_to_ignore_3d )
    pose_3d = np.zeros((enc_in.shape[0],96, out_mean.shape[-1]))

    for j in range(out_mean.shape[-1]):
        pose_3d[:, :, j] = unNormalizeData( out_mean[:, :, j], data_mean_3d, data_std_3d, dim_to_ignore_3d )

    pose_3d_ = copy.deepcopy(pose_3d)
    all_poses_3d.append(pose_3d_)

    # Keep only the relevant dimensions
    dtu3d = np.hstack( (np.arange(3), dim_to_use_3d) ) if not(FLAGS["predict_14"]) else  dim_to_use_3d

    dec_out = dec_out[:, dtu3d]
    pose_3d = pose_3d[:, dtu3d,:]

    assert dec_out.shape[0] == FLAGS["batch_size"]
    assert pose_3d.shape[0] == FLAGS["batch_size"]

    if FLAGS["procrustes"]:
      # Apply per-frame procrustes alignment if asked to do so
      for j in range(FLAGS["batch_size"]):
          for k in range(model.num_models):
            gt  = np.reshape(dec_out[j,:],[-1,3])
            out = np.reshape(pose_3d[j,:, k],[-1,3])
            _, Z, T, b, c = procrustes.compute_similarity_transform(gt,out,compute_optimal_scale=True)
            out = (b*out.dot(T))+c

            pose_3d[j, :, k] = np.reshape(out,[-1,17*3] ) if not(FLAGS["predict_14"]) else np.reshape(pose_3d[j,:, k],[-1,14*3] )

    # Compute Euclidean distance error per joint
    sqerr = (pose_3d - np.expand_dims(dec_out,axis=2))**2 # Squared error between prediction and expected output
    dists = np.zeros((sqerr.shape[0], n_joints, sqerr.shape[2])) # Array with L2 error per joint in mm

    for m in range(dists.shape[-1]):
      dist_idx = 0
      for k in np.arange(0, n_joints*3, 3):
        # Sum across X,Y, and Z dimenstions to obtain L2 distance
        dists[:,dist_idx, m] = np.sqrt( np.sum( sqerr[:, k:k+3,m], axis=1 ))

        dist_idx = dist_idx + 1

    all_dists.append(dists)
    assert sqerr.shape[0] == FLAGS["batch_size"]

  step_time = (time.time() - start_time) / nbatches
  loss      = loss / nbatches

  all_dists = np.vstack( all_dists )
  aver_minerr = np.mean(np.min(np.sum( all_dists, axis=1),axis=1))/n_joints

  return aver_minerr, step_time, loss

In [37]:
def evaluate_orig(
    test_set_2d, test_set_3d,
    data_mean_2d, data_mean_3d,
    data_std_2d, data_std_3d,
    dim_to_use_2d, dim_to_use_3d,
    dim_to_ignore_2d, dim_to_ignore_3d):

    actions = define_actions( FLAGS["action"] )

    if FLAGS["evaluateActionWise"]:

      cum_err = 0           # select the mixture model which has mininum error
      for action in actions:

        # Get 2d and 3d testing data for this action
        action_test_set_2d = get_action_subset( test_set_2d, action )
        action_test_set_3d = get_action_subset( test_set_3d, action )
        encoder_inputs, decoder_outputs = model.get_all_batches( action_test_set_2d, action_test_set_3d, FLAGS["camera_frame"], training=False)

        act_err, step_time, loss = evaluate_batches( sess, model,
                                                      data_mean_3d, data_std_3d,
                                                      data_mean_2d, data_std_2d,
                                                      encoder_inputs, decoder_outputs)
        
        print(action, " ", act_err)

        cum_err = cum_err + act_err

    print(cum_err / float(len(actions)))
    

## Experiments

### Random Sample

In [9]:
random_data_2d = np.random.rand(64, 32)
random_data_3d = np.random.rand(64, 48)

evaluate(random_data_2d, random_data_3d)

Creating 2 bi-layers of 1024 units.
0.3498102956277592


### 3DPW Attempt

In [22]:
data_3dpw = np.load('data/3dpw_combined_actors_v2.npz', allow_pickle=True, encoding='latin1')['data'].item()

indices_to_select = [0, 2, 5, 8, 1, 4, 7, 6, 12, 15, 16, 18, 20, 17, 19, 21]

data_count =  data_3dpw['test']['combined_3d_cam'].shape[0]

data_3d = data_3dpw['test']['combined_3d_cam'][:, indices_to_select, :].reshape((data_count, 48))
data_3d = data_3d - np.tile(data_3d[:, :3], [1, 16])
data_2d = data_3dpw['test']['combined_2d'][:, indices_to_select, :].reshape((data_count, 32)) 

In [23]:
data_3d = normalize_data(data_3d, mean_3d, std_3d)
data_2d = normalize_data(data_2d, mean_2d, std_2d)

In [24]:
evaluate(data_2d, data_3d, [mean_2d, mean_3d], [std_2d, std_3d])

Creating 2 bi-layers of 1024 units.
390.89428236198


### GPA Attempt

In [25]:
data_gpa = np.load('data/gpa_xyz_projected_wc_v3.npz', allow_pickle=True, encoding='latin1')['data'].item()

indices_to_select = [0, 24, 25, 26, 29, 30, 31, 2, 5, 6, 7, 17, 18, 19, 9, 10, 11]
indices_to_sort = [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16]

data_count = data_gpa['test']['3d'].shape[0]

data_3d = data_gpa['test']['3d'][:, indices_to_select, :][:, indices_to_sort, :].reshape((data_count, 48))*10
data_3d = data_3d - np.tile(data_3d[:, :3], [1, 16])
data_2d = data_gpa['test']['2d_projected'][:, indices_to_select, :][:, indices_to_sort, :].reshape((data_count, 32))

evaluate(data_2d, data_3d)

Creating 2 bi-layers of 1024 units.
173.7848154533636


In [48]:
data_h36m = h5py.File('data/h36m_s1_directions1.h5', 'r')

indices_to_select = [0, 1, 2, 3, 6, 7, 8, 12, 13, 15, 17, 18, 19, 25, 26, 27]

data_count = data_h36m['pose']['3d-univ'].shape[0]

data_3d = np.array(data_h36m['pose']['3d-univ'][:, indices_to_select, :]).reshape((data_count, 48))
# data_3d = data_3d - np.tile(data_3d[:, :3], [1, 16])
data_2d = np.array(data_h36m['pose']['2d'][:, indices_to_select, :]).reshape((data_count, 32))

print(data_3d[0, :].reshape((16, 3)))
print(data_2d[0, :].reshape((16, 2)))

# evaluate(data_2d, data_3d)

[[-176.73076784 -321.04861816 5203.88206303]
 [ -52.96191118 -309.7044902  5251.08279046]
 [-155.64155821   73.07175588 5448.80702584]
 [ -29.83157265  506.78445233 5400.13833872]
 [-300.49984317 -332.39276616 5156.68125222]
 [-258.24048857   99.60905053 5244.68147203]
 [-209.48436389  548.83382497 5290.76367197]
 [-109.15762285 -529.7281668  5123.8906273 ]
 [-140.19117593 -780.12137495 5074.60478778]
 [-118.93483197 -970.22827344 5058.5990502 ]
 [-259.08997397 -690.13358895 5050.59205089]
 [-370.67089216 -448.59930095 5134.17726128]
 [-462.28663516 -290.82947286 5307.6274481 ]
 [ -19.76034113 -716.91810277 5140.27255285]
 [  35.79161007 -470.14491849 5257.73845884]
 [  13.89246482 -279.85293245 5421.06854165]]
[[473.68356 444.9424 ]
 [500.9961  448.02988]
 [479.83926 530.78564]
 [506.21838 622.56885]
 [445.9001  441.81586]
 [456.18906 537.1581 ]
 [467.30923 633.76935]
 [488.18674 397.43405]
 [481.02847 340.39694]
 [485.76895 297.57162]
 [454.01608 359.75955]
 [430.05878 415.7349 ]
 [4

### H36M Verification

In [19]:
TRAIN_SUBJECTS = [1,5,6,7,8]
TEST_SUBJECTS  = [9,11]

actions = define_actions(FLAGS['action'])

orig_h36m_train_3d = load_h36m_dataset('../../Implementations/GenMultiHypothesis/data/h36m/', TRAIN_SUBJECTS, actions, 3)
orig_h36m_test_3d = load_h36m_dataset('../../Implementations/GenMultiHypothesis/data/h36m/', TEST_SUBJECTS, actions, 3)

positions/Sitting*.h5
../../Implementations/GenMultiHypothesis/data/h36m/S6/MyPoses/3D_positions/Sitting 1.h5
../../Implementations/GenMultiHypothesis/data/h36m/S6/MyPoses/3D_positions/Sitting 2.h5
Reading subject 6, action SittingDown
../../Implementations/GenMultiHypothesis/data/h36m/S6/MyPoses/3D_positions/SittingDown*.h5
../../Implementations/GenMultiHypothesis/data/h36m/S6/MyPoses/3D_positions/SittingDown 1.h5
../../Implementations/GenMultiHypothesis/data/h36m/S6/MyPoses/3D_positions/SittingDown.h5
Reading subject 6, action Smoking
../../Implementations/GenMultiHypothesis/data/h36m/S6/MyPoses/3D_positions/Smoking*.h5
../../Implementations/GenMultiHypothesis/data/h36m/S6/MyPoses/3D_positions/Smoking.h5
../../Implementations/GenMultiHypothesis/data/h36m/S6/MyPoses/3D_positions/Smoking 1.h5
Reading subject 6, action Waiting
../../Implementations/GenMultiHypothesis/data/h36m/S6/MyPoses/3D_positions/Waiting*.h5
../../Implementations/GenMultiHypothesis/data/h36m/S6/MyPoses/3D_positions/

In [11]:
print(orig_h36m_test_3d[(9, 'Directions', 'Directions 1.h5')].shape)

(2356, 96)


In [20]:
rcams = cameras.load_cameras('data/h36m_cameras.h5')

orig_h36m_train_cam = transform_world_to_camera(orig_h36m_train_3d, rcams)

orig_h36m_train_cam, _ = postprocess_3d(orig_h36m_train_cam)

complete_train = copy.deepcopy(np.vstack(orig_h36m_train_cam.values()))

mean_3d, std_3d, dti_3d, dtu_3d = normalization_stats(complete_train, 3)

# orig_h36m_train_cam = normalize_data(complete_train, mean_3d, std_3d)

In [21]:
orig_h36m_test_cam = transform_world_to_camera(orig_h36m_test_3d, rcams)

print(orig_h36m_test_cam[(9, 'Directions', 'Directions 1.54138969.h5')].shape)

orig_h36m_test_cam, _ = postprocess_3d(orig_h36m_test_cam)

complete_test = copy.deepcopy(np.vstack(orig_h36m_test_cam.values()))

print(orig_h36m_test_cam[(9, 'Directions', 'Directions 1.54138969.h5')].shape)

orig_h36m_test_cam = normalize_data(orig_h36m_test_cam, mean_3d, std_3d, dtu_3d)

(2356, 96)
(2356, 96)


In [22]:
orig_h36m_train_2d = project_to_cameras(orig_h36m_train_3d, rcams)

complete_train_2d = copy.deepcopy(np.vstack(orig_h36m_train_2d.values()))

mean_2d, std_2d, dti_2d, dtu_2d = normalization_stats(complete_train_2d, dim=2)

# orig_h36m_train_2d = normalize_data(complete_train_2d, mean_2d, std_2d)

In [23]:
orig_h36m_test_2d = project_to_cameras(orig_h36m_test_3d, rcams)

orig_h36m_test_2d = normalize_data(orig_h36m_test_2d, mean_2d, std_2d, dtu_2d)

In [17]:
indices_to_select = [0, 1, 2, 3, 6, 7, 8, 12, 13, 15, 17, 18, 19, 25, 26, 27]

mod_2d = orig_h36m_test_2d[(9, 'Directions', 'Directions 1.54138969.h5')].reshape((-1, 32, 2))
mod_2d = mod_2d[:, indices_to_select, :]

mod_3d = orig_h36m_test_cam[(9, 'Directions', 'Directions 1.54138969.h5')].reshape((-1, 32, 3))
mod_3d = mod_3d[:, indices_to_select, :]

In [22]:
mod_mu_2d = mean_2d.reshape((32, 2))[indices_to_select, :].reshape((32))
mod_mu_3d = mean_3d.reshape((32, 3))[indices_to_select, :].reshape((48))
mod_std_2d = std_2d.reshape((32, 2))[indices_to_select, :].reshape((32))
mod_std_3d = std_3d.reshape((32, 3))[indices_to_select, :].reshape((48))

In [23]:
evaluate(
    mod_2d.reshape((-1, 32)),
    mod_3d.reshape((-1, 48)),
    [mod_mu_2d, mod_mu_3d],
    [mod_std_2d, mod_std_3d]
)

Creating 2 bi-layers of 1024 units.
199.0036622693718


In [38]:
evaluate_orig(
    orig_h36m_test_2d, orig_h36m_test_cam,
    mean_2d, mean_3d,
    std_2d, std_3d,
    dtu_2d, dtu_3d,
    dti_2d, dti_3d
)

Directions   43.10767701930445
Discussion   51.05013986771212
Eating   51.724349459074
Greeting   49.84631152856204
Phoning   57.32854292441411
Photo   64.59227430961076
Posing   47.56292327853188
Purchases   47.49338592765643
Sitting   64.01063001842849
SittingDown   71.56349413179795
Smoking   55.90719920972362
Waiting   50.81513474262019
WalkDog   57.66722607785159
Walking   41.08270919073637
WalkTogether   44.42559820420118
53.21183972601502
